In [2]:
# switch to the project directory
%cd ..
# working directory should be ../FSE

c:\Users\admin\Desktop\research\FSE


In [7]:
import pandas as pd
import numpy as np
from pdi.data.constants import INPUT_PATH, CSV_DELIMITER, MISSING_VALUES

data = pd.read_csv(INPUT_PATH, sep=CSV_DELIMITER, index_col=0)
for column, val in MISSING_VALUES.items():
    data.loc[data[column] == val, column] = np.NaN

C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [15]:
print(data.shape)

(2751934, 22)


In [26]:
classes = data["fPdgCode"].value_counts()
print(classes)
chosen_classes=[-211, 211, 321, -321, 2212, -2212]

num_chosen = classes[chosen_classes].sum()
print(num_chosen)
print(num_chosen / data.shape[0])

-211           1201576
 211           1199557
 321             93982
-321             90488
 2212            55748
-2212            50057
 11              21840
-11              20978
 13               8880
-13               8825
 1000010030          2
 1000010020          1
Name: fPdgCode, dtype: int64
2691408
0.9780060132256079


In [8]:
nulls = data.isnull().sum()
print(nulls)

fTPCSignal                  0
fTRDSignal            1047980
fTRDPattern           1047980
fTOFSignal            1692076
fBeta                 1692076
fP                          0
fPt                         0
fPx                         0
fPy                         0
fPz                         0
fSign                       0
fX                          0
fY                          0
fZ                          0
fAlpha                      0
fTrackType                  0
fTPCNClsShared              0
fDcaXY                      0
fDcaZ                       0
fPdgCode                    0
fIsPhysicalPrimary          0
P                           0
dtype: int64


In [19]:
nulls_rows = data.isnull().value_counts()
print(nulls_rows)

fTPCSignal  fTRDSignal  fTRDPattern  fTOFSignal  fBeta  fP     fPt    fPx    fPy    fPz    fSign  fX     fY     fZ     fAlpha  fTrackType  fTPCNClsShared  fDcaXY  fDcaZ  fPdgCode  fIsPhysicalPrimary  P    
False       False       False        False       False  False  False  False  False  False  False  False  False  False  False   False       False           False   False  False     False               False    1021933
            True        True         True        True   False  False  False  False  False  False  False  False  False  False   False       False           False   False  False     False               False    1010055
            False       False        True        True   False  False  False  False  False  False  False  False  False  False   False       False           False   False  False     False               False     682021
            True        True         False       False  False  False  False  False  False  False  False  False  False  False   False       Fals

In [24]:
all_nulls = data.isnull().any(axis=1).sum()
print(all_nulls)
print(all_nulls/data.shape[0])

1730001
0.6286491609173767
